In [1]:
import os, random, logging
import utils.auxua as aux
import utils.watch as watch
import utils.tf_formh5 as tf_formh5
import numpy as np

''' tf log level & neptune'''
tf_formh5.set_tf_loglevel(logging.ERROR)

Num GPUs Available:  4
PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')
PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU')


/home/jtstudents/anaconda3/envs/zugpu/lib/python3.8/site-packages/neptune/internal/backends/hosted_client.py:48: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version


In [2]:
'''get_results_from_txt'''

res_list_full,res_list_max,res_list_fn,res_list_path,res_list_labels,res_path,res_model_fn,res_model_fn_strap =\
aux.get_results_from_txt(aux.RSLT_PATH,printt=True)

res_list={
    'full':res_list_full,
    'max':res_list_max,
    'videoname':res_list_fn,
    'videopath':res_list_path,
    'labels':res_list_labels,
    'txt_path':res_path,
    'txt_modelname':res_model_fn,
    'txt_modelname_strap':res_model_fn_strap        
}

for i in range(len(res_list['txt_modelname'])):print(i,res_list['txt_modelname'][i])

#res_list_1 = aux.get_results_from_txt("/raid/DATASETS/.zuble/vigia/zu++/model/rslt/1671124367.446378_3gelu_adamamsgrad_0_4000_weights.txt")

FOLDER

results for 3gelu_sgd_0_4000_weights
	ACCURACY   0.81875
	PRECISION (%% of True 1 out of all Positive predicted)  0.8867102
	RECALL (%% of True Positive out of all actual anomalies)  0.814
	AUPRC ( AreaUnderPrecisionRecallCurve ) 0.9295 
	 AUC-ROC 0.8896 
	F1_SCORE (harmonic mean of precision and recall)  0.8488007709181216

results for 3gelu_adam_0_4000_weights
	ACCURACY   0.77125
	PRECISION (%% of True 1 out of all Positive predicted)  0.9116883
	RECALL (%% of True Positive out of all actual anomalies)  0.702
	AUPRC ( AreaUnderPrecisionRecallCurve ) 0.9290 
	 AUC-ROC 0.8835 
	F1_SCORE (harmonic mean of precision and recall)  0.7932203158709636

results for 3gelu_adamamsgrad_0_4000_weights
	ACCURACY   0.7875
	PRECISION (%% of True 1 out of all Positive predicted)  0.9166667
	RECALL (%% of True Positive out of all actual anomalies)  0.726
	AUPRC ( AreaUnderPrecisionRecallCurve ) 0.9398 
	 AUC-ROC 0.9044 
	F1_SCORE (harmonic mean of precision and recall)  0.8102678471795525

res

In [19]:
def init_watch_reslist(res_list, txt_i,watch_this):
    
    def get_fn_index_from_list(txt_i):
        '''retrives all video indexs from a rslt.txt that are false negatives'''
        
        print("\nget_fn_index_from_list ",res_list['txt_modelname'][txt_i])

        labels_indexs_fn={'AA':[],'B1':[],'B2':[],'B4':[],'B5':[],'B6':[],'G':[],'BG':[]}

        for video_j in range(len(res_list['videoname'])):
            if 'label_A' not in res_list['videoname'][video_j][txt_i]:
                labels_indexs_fn['AA'].append(video_j)
                if float(res_list['max'][video_j][txt_i])<=0.5: 
                    labels_indexs_fn['BG'].append(video_j)
                    if 'label_B1' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['B1'].append(video_j)
                    if 'label_B2' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['B2'].append(video_j)
                    if 'label_B4' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['B4'].append(video_j)
                    if 'label_B5' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['B5'].append(video_j)
                    if 'label_B6' in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['B6'].append(video_j)
                    if 'label_G'  in res_list['videoname'][video_j][txt_i]:labels_indexs_fn['G'].append(video_j)   

        return labels_indexs_fn
    
    as_total = watch.get_as_total_from_res_list(res_list,txt_i)
    
    labels_indexs_fn=get_fn_index_from_list(txt_i)
    print(  '\nFALSE NEGATIVES',\
            '\n  B1 FIGHT',  len(labels_indexs_fn['B1']),\
            '\n  B2 SHOOT',  len(labels_indexs_fn['B2']),\
            '\n  B4 RIOT',   len(labels_indexs_fn['B4']),\
            '\n  B5 ABUSE',  len(labels_indexs_fn['B5']),\
            '\n  B6 CARACC', len(labels_indexs_fn['B6']),\
            '\n  G EXPLOS',len(labels_indexs_fn['G']),\
            '\n  B+G',len(labels_indexs_fn['BG']),\
            '\nAA ALL ANOMALIES',len(labels_indexs_fn['AA']),'\n')
    
    print('set to watch this',watch_this)
    for labels_2_watch in watch_this:
        print(labels_2_watch)
        for i in range(len(labels_indexs_fn[labels_2_watch])):
            video_j = labels_indexs_fn[labels_2_watch][i]
            watch.cv2_test_from_aslist(res_list,as_total[video_j],video_j,txt_i)

In [20]:
'''
False Negatives 
    A  NORMAL
    B1 FIGHT
    B2 SHOOTING
    B4 RIOT
    B5 ABUSE
    B6 CAR ACCIDENT
    G  EXPLOSION 
    BG ALL FN
    
    AA ALL ANOM
'''

init_watch_reslist(res_list,9,watch_this=['BG'])

as_total from /raid/DATASETS/.zuble/vigia/zu++/model/rslt/1677498953.2416248_relu_sgd_0_4000_weights.txt

get_fn_index_from_list  1677498953.2416248_relu_sgd_0_4000_weights

FALSE NEGATIVES 
  B1 FIGHT 27 
  B2 SHOOT 10 
  B4 RIOT 16 
  B5 ABUSE 4 
  B6 CARACC 6 
  G EXPLOS 5 
  B+G 68 
  AA 500 

set to watch this ['B6']
B6

res_list['full'] for video  88 | txt 9 ['1', '1225', '0.06356531']

res_list['full'] for video  496 | txt 9 ['1', '840', '0.019077143']

res_list['full'] for video  520 | txt 9 ['1', '2522', '0.37061223']

res_list['full'] for video  561 | txt 9 ['1', '2186', '0.19388032']

res_list['full'] for video  677 | txt 9 ['1', '1226', '0.48979995']

res_list['full'] for video  731 | txt 9 ['1', '2522', '0.00054345495']
